# Run Metadynamics Simulations

In this example we will prepare a Well-Tempered Metadynamics (WT-Metad) simulation on torsional angles of an organic molecule. 

In particular we will see how to:  
1) Load an existing project
2) Do a basic analysis of the unbiased MD simulation in order to estimate the parameters for the WT-Metad simulation
3) Prepare the WT-Metad simulation



## 0. Load the necessary modules:



In [1]:
from sim_launch_py.classes import Project
from sim_launch_py.external import gaussianFit
from sim_launch_py.plumed import write_plumed_file as wrtplumed
from copy import deepcopy
import numpy as np

## 1. Load the project

The first step is to load the state of the project with the **Project.load_project()** method.

In [2]:
loadpath='/home/matteo/Work/organic_molecules_simulations/examples/sample_metad'
p=Project.load_project(loadpath)


Loading Project Name: sample_metad



## 2. Analyse previous simulations

### 2.1 Recover the simulation of interest

In this example we will use data from the last simulation from the saved state of the project. We can get the last simulation from the list of simulations of the system either by its name (e.g. 'md') or by its index. Here we use its name.

In [3]:
s=p.systems[-1]
sim=s.find_simulation_by_name('md')


### 2.2 Recover position and width of the states observed in unbiased MD

For each dihedral angle in the system we read the corresponding column in the data file (e.g. the COLVAR file from the 'md' simulation) and fit the distributions to sums of gaussians curves with the **gaussianFit.gaussianFit()** external method.  
From the fit we obtain the centers of the distributions and their sigma parameter.  

In [4]:
ndih=len(sim.cvs)
centers,sigmas=[],[]

for col in range(ndih):

    cen,sig=gaussianFit.gaussianFit(sim.path+'/COLVAR',datacol=col+1)

    centers.append(cen)
    sigmas.append(sig)


## 3. Prepare a Metad simulation

### 3.1 Define the simulation *dict* and add the simulation to the system

Define the simulation _dict_ as done for the simulations in the previous example on the Study Torsional Angles of Organic Molecules.  
Then add the simulation to the system.


In [5]:
    
metad_dict={'path_mdp': '/home/matteo/Work/organic_molecules_simulations/sim_launch_py/mdp/mdparrinello.mdp',
            'maxwarn':2, 'coordinates':'{}/{}.gro'.format(s.simulations[-1].path,s.simulations[-1].name),
            'nsteps':5000000, 'plumed':'plumed.dat'}

s.add_simulation('metad','md',simulation_dict=metad_dict)
newsim=s.find_simulation_by_name('metad')

Simulation metad(type md) will start from configuration ./Systems/ccdc-xxxii_toluene/md/md.gro.
Added simulation metad (4) to system ccdc-xxxii_toluene (8).



### 3.2 Add CVs and Metadynamics bias

For each dihedral angle of the MD simulation, we create the corresponding new CV in the new WT-MetaD simulation (we can directly copy it with **copy.deepcopy()**) and create the bias *dict* that is used to define the bias applied to the CV in the simulation.

A typical metadynamics *dict* will look like:

```
metadbias_dict={'sigma':sigma,        # one SIGMA parameter vale or a list of values 
                                        (one per each cv, float)
                'height':height,      # HEIGHT parameter for the METAD directive in plumed
                'biasfactor':bf:      # BIASFACTOR parameter for the METAD directive (float). If omitted,  
                                        it will assume standard Metadynamics (constant height)
                'grid_min':gmin       # GRID_MIN values (one or a list of values, either float or str)
                'grid_max':gmax       # GRID_MAX values (one or a list of values, either float or str)
                'grid_spacing':gspace # GRID_SPACING values (one or a list of values, either float or str)
                'pace':pace           # PACE value (int)
                'hills_file':hills    # HILLS value (str)
                'temperature':temp    # TEMP value (only for WT-Metad, float)
}  
```

Note that, in this example, for each dihedral angle of the molecule, we use the smallest sigma value from the fit of the distribution as an estimate for the SIGMA parameter of the concurrent WT-Metad simulation.

In [6]:
sim=s.find_simulation_by_name('md')

for idih,dih_sigma in enumerate(sigmas):
    
    sig=np.round(min(dih_sigma)/2,decimals=3)

    # duplicate dihedral cvs used in MD simulation
    newcv=deepcopy(sim.cvs[idih])
    newsim.cvs.append(newcv)

    metadbias_dict={'sigma':sig,
                    'height':2.5,
                    'biasfactor':5,
                    'grid_min':'-pi',
                    'grid_max':'pi',
                    'grid_spacing':np.round(np.pi/180*2,decimals=4),
                    'pace':500,
                    'hills_file':'HILLS_dih_{}'.format(idih)}

    newsim.add_bias('metad_dih_{}'.format(idih),'METAD',newsim.cvs[-1].name,bias_dict=metadbias_dict)

wrtplumed('{}/plumed.dat'.format(newsim.path),newsim,colvar='COLVAR') #### STAMPARE ANCHE I METAD.BIAS


## 4. Prepare simulation scripts

Finally we prepare the job script to launch the simulation. We set the state of the previous simulations of the system to 'End' in order to write the commands regarding only the new WT-Metad simulation in the job script.

***This will change in future and will be automated***


In [7]:
ntot_sims=len(s.simulations)
for isim in range(ntot_sims-1):
    s.simulations[isim].state='Ended'

myriad_dict={'wallclock':'12:00:00',
             'job_name':s.name,
             'mpi':12,
             'omp':6,
             'gmx_bin':'gmx_mpi'}

s.create_run_script('run_metad.myriad',platform='myriad',platform_dict=myriad_dict)

Written run script file in ./Systems/ccdc-xxxii_toluene/run_metad.myriad


In [8]:
newsim.biases[0].__dict__

{'_name': 'metad_dih_0',
 '_biastype': 'METAD',
 'directive': 'METAD ...\n    ARG=dih_0,\n    FILE=HILLS_dih_0\n    SIGMA=0.284,\n    HEIGHT=2.5\n    PACE=500\n    BIASFACTOR=5\n    TEMP=300.0\n    GRID_MIN=-pi,\n    GRID_MAX=pi,\n    GRID_SPACING=0.0349,\n... METAD\n'}

run `gmx`